# Floating Point Representation in GA

In [4]:
import random
import numpy as np

Floating point representation
 
A representation of the numer 12.0000 can be in the following ways
 
 1. chromosome = "120000" meaning 12.0000 
 2. [1,2,0,0,0,0] 
 3. 12.00000

# Converting from binary to float and float to binary in python
We can also represent floating points as binary 

In [5]:
# float to binary using numpy here the minimum is 16 bit representation

a = bin(np.float16(12.5).view('H'))[2:].zfill(16)

# binary to float

import struct
b = struct.pack("H",int(a, 2))
c = np.frombuffer(b , dtype =np.float16)[0]

a, b, c

('0100101001000000', b'@J', 12.5)

In [6]:
# using struct , here the minimum is 32 bit representation

import struct

def float_to_bin(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

a = float_to_bin(20.1)
b = bin_to_float(a)

a, b, len(a)

('01000001101000001100110011001101', 20.100000381469727, 32)

# CrossOver methods for floating point representation

All the crossover methods for binary representation can be used for floating point except for multiparent. In floating point representation, each number is a decimal. 

# 1. Linear CrossOver Operator
Given a group of chromosomes, three parents are selected and crossover is carried to produce three offsprings and the best to might be selected depending on what is desired. The crossover is as follows:
* $(X_1 + X_2) , (1.5*X_1 - 0.5*X_2), (-0.5*X_1 + 1.5*X_2)$

In [7]:
# linear crossover operator 

def lco(A,B):
    
    A0 = A + B
    B0 = (1.5*A) - (0.5*B)
    C0 = (-0.5*A) + (1.5*B)
    
    return A0, B0, C0


In [8]:
A = 2.5
B = 3.6
C = lco(A, B)

A, B, C

(2.5, 3.6, (6.1, 1.95, 4.15))

# 2. Directional Heuristic crossover

one offspring is created from two parents using

$ X _{i,j}' = U(0, 1)*(X_{2j} − X_{1j}) + X_{2j}$

subject to the constraint that parent X2 cannot be worse than parent X1. i is the parent number and j is a dimension or gene position in chromosome if an array is used 

In [9]:
def dhc(A,B):
    
    return ( random.random()*(A-B) ) + A  # given that A is better than B 

In [10]:
C = dhc(A,B)

A,B,C

(2.5, 3.6, 1.5595002110246532)

In [11]:
# if A and B are represented by a string of chromosomes

def Sdhc(A, B):
    A0 = []
    
    for i in range(len(A)):
        A0.append((random.random()*(A[i]-B[i]) ) + A[i])   # given that A is better than B
    
    return A0

In [12]:
SA = [0.5, 0.4, 0.2, 2.3]
SB = [1.5, 2.4, 0.8, 0.3]

SA0 = Sdhc(SA,SB)

SA0, SA, SB

([-0.11135043845498516,
  -0.033043998635012506,
  0.04170922450071418,
  2.576709196137059],
 [0.5, 0.4, 0.2, 2.3],
 [1.5, 2.4, 0.8, 0.3])

# 3. Arithmetic CrossOver
It's a multiparent recombination strategy that takes a weighted average over two or more parents. One offspring is generated using
* $ X_{i,i}' = \sum_{l=1}^{n}\gamma_l X_{l,j} $ where n is the number of parents and
* $ \sum_{l=1}^{n}\gamma_l = 0 $

In [14]:
gammas = [random.random() for i in range(3)]
gamma = [i/sum(gammas) for i in gammas]

gammas, gamma, sum(gamma), sum(gammas)

([0.7077179603499998, 0.9884185234708575, 0.7053846686734327],
 [0.2946957013536788, 0.4115801863515785, 0.2937241122947427],
 1.0,
 2.40152115249429)

In [17]:
# the gammas can be either uniform or random 

def ac(parents):
    """ parents is the number of parents"""
    n = len(parents)
    gammas = [random.random() for i in range(n)]
    gammas = [i/sum(gammas) for i in gammas]
    A0 = 0
    
    for i in range(n):
        A0 += (gammas[i]*parents[i])
    
    return A0
    

In [18]:
A , B , C, D = (2.4, 5.3, 6.2, 1.7)
E = [A , B , C, D]

A0 = ac(E)

A0, A, B, C, D, E

(4.295384169960936, 2.4, 5.3, 6.2, 1.7, [2.4, 5.3, 6.2, 1.7])

# 4. Blend CrossOver (BLX-$\alpha$)
This crossover method is more exploratory

* $ X_{i,i}' = (1- \gamma_j)*X_{1,j} + \gamma_j*X_{2,j} $
* where $\gamma_j = (1 - 2\alpha) U(0,1) - \alpha $ 

The value of $\alpha = 0.5 $ works and is commonly used. BLX-$\alpha$ assumes that X1 < X2

* The uniform operator randomly picks a number in the range
$[X_{1,j} − α(X_{2,j} − X_{1,j}), X_{2,j} + α(X_{2,j} − X_{1,j})]$

In [49]:
def blx(A, B, alpha = 0.4):
    # ensure A is less than B
    
    U = (A - (alpha*B - A) , B + (alpha*B - A) )
    gamma = (1 - (2*alpha) )*( random.uniform(U[0], U[1]) ) - alpha
    #gamma1 = (1 - (2*alpha) )*( random.uniform(0, 1) ) - alpha
    
    A0 = ((1 - gamma)*A ) + ( gamma*B )
    #A1 = ((1 - gamma1)*A ) + ( gamma1*B )
    
    return A0 #, A1 

In [51]:
U = (A - (0.5*B - A) , B + (0.5*B - A) )
C = blx(A, B)

C, A, B, U, random.uniform(U[0], U[1])

(2.959737935364127, 2.4, 5.3, (2.15, 5.55), 4.827423151721749)

# 5. Geometric CrossOver
Produces one offspring from 2 parents 
* $ X_{i,j}' = (X_{1,j}*X_{2,j})^{0.5} $

In [52]:
def gc(A,B):
    
    return (A*B)**0.5

In [53]:
C = gc(A,B)

C, A, B

(3.5665109000254014, 2.4, 5.3)

# 6. Geometric Multiparent CrossOver
Given n parents the generalization of the geometric crossover is as follows
* $X_{i,j}' = X_{1,j}^\alpha_1*X_{2,j}^\alpha_2*...*X_{n,j}^\alpha_n$ where n is the number of parents and
* $ \sum_{l=1}^{n}\alpha_l = 1 $ 

In [70]:
def gmc(parents):
    """parents is a list of list with parents """
    n = len(parents)
    alphas = [random.random() for i in range(n)]
    alphas = [i/sum(alphas) for i in alphas]
    #print(alphas)
    A0 = parents[0]*alphas[0]
    
    for i in range(1,n):
        #print (A0)
        A0 *= parents[i]*alphas[i]
        
    return A0

In [71]:
A , B , C, D = (2.4, 5.3, 6.2, 1.7)
E = [A , B , C, D]

A0 = gmc(E)

A0, A, B, C, D, E

(0.36304611468243886, 2.4, 5.3, 6.2, 1.7, [2.4, 5.3, 6.2, 1.7])

# 7. Simulated Binary Crossover (SBX)
simulated binary crossover (SBX) simulates the behavior of the one-point crossover operator for binary representations. Two parents, x1  and x2 are used to produce two offspring, where for j = 1, . . . , nx  j is dimension

* $x_{1j}' = 0.5[(1 + γ_j)x_{1j} + (1 − γ_j)x_{2j}]$ 
* $x_{2j}' = 0.5[(1 − γ_j)x_{1j} + (1+γ_j)x_{2j}]$
* $γ_j = (2r_j)^{\frac{1}{η+1}} if r_j <= 0.5 $ Else $γ_j = (\frac{1}{2(1-r_j)})^{\frac{1}{η+1}} if r_j >= 0.5 $
* where $r_j = U (0,1) $ and η preferably 1.

For large values of η there is a higher probability thatoffspring will be created near the parents. For small η values, offspring will be more distant from the parents.

In [104]:
def sbc(A,B):
    eta = 1
    r  = random.random()
    
    if r <= 0.5:
        gamma = (2*r)**(1/(eta+1))
        #print("HErE.....1:  ", r)
    else:
        gamma = ( 1/(2*(1-r)) )**(1/(eta+1))
        #print("HErE.....2:  ", r)
        
    A0 = 0.5*( (1 + gamma)*A + (1 - gamma)*B )
    B0 = 0.5*( (1 - gamma)*A + (1 + gamma)*B )
    
    return A0, B0

In [105]:
C, D = sbc(A, B)

C, D, A, B

(2.4298508894190243, 5.2701491105809755, 2.4, 5.3)

# The next set of Crossover methodologies are Exclusively multiparent crossover
The main objective of these multiparent operators is to intensify the explorative capabilities compared to two-parent operators. By aggregating information from multiple parents, more disruption is achieved with the resemblance between offspring and parents on average smaller compared to two-parent operators.

# 8. Unimodal Distribution Operator (UNDX)
Two offsprings are generated from two parents. It can also be generalized for more than three parents up n-1. for the parents selected, their centre of mass (mean) is computed as follows

* $Xc = \sum_{l=1}^{n-1}X_{l,j} $ n-1 is the number of parents

from the mean, n-1 directional vectors are computed from all the parents using

* $ d_l = X_l - Xc$

Using the direction vectors, the direction cosines are computed as

* $ e_l = \frac{d_l}{|d_l|}$ where $|d_l|$ is the length of vector $d_l$

a random parent x is selected from the set of parents and an ortogonal vector 

* $x - Xc$  orthogonal to $e_l$ and
* $\delta = |x - Xc|$. the distance between x and Xc

offsprings are generated Using 

* $ X_i' = Xc + \sum_{l=1}^{n-1}N(0, \sigma_1^2)|d_l|e_l + \sum_{n}^{n_s}N(0, \sigma_2^2)\delta e_l$


* where $\sigma_1 = \frac{1}{\sqrt{n-2}}$ and $\sigma_2 = \frac{0.35}{\sqrt{n_s-n-2}} $ 



# 9. Simplex CrossOver Operator (SPX)
selects n > 2 parents, and determines the best and worst parent, say x1 and x2 respectively. The center of mass, Xc is computed over the selected parents, but with x2 excluded. One offspring is generated using

$ x' = Xc + (x1 − x2) $

# 10. Expanded Simplex CrossOver Operator
selecting n = nx + 1 parents independent from one another for an nx-dimensional search space. These n parents form a simplex. The simplex is expanded in each of the n directions, and offspring sampled from the expanded simplex. the expanded simplex is defined by the points
* $ (1 + γ)(x_l − Xc)$ l = 1...n, and γ >= 0

The offspring is choosen by random sampling 

# 11. Parent-Centric CrossOver (PCX)
Its a variation of the UNDX operator. Instead of generating offspring around the center of mass of the selected parents, offspring are generated around selected parents. PCX selects n parents and computes their center of mass, Xc. For each offspring to be generated, one parent is selected uniformly from the n parents. A direction vector is
calculated for each offspring as

* $d_i = x_i − Xc$ where $x_i$ is a randomly generated parent 

From the remaining n-1 parents, perpendicular distances to $d_l$ is $\delta_l$ are calculated and the average over this distances calculated.

$\delta = |x−Xc| $

$ \delta' = \frac{sum_{l=1, l!=i}^{n-1}\delta_l}{n-1} $

offsprings are generated as follows

$ X_i' = X_i + N(0, \sigma_1^2)|d_i| + \sum_{l=1, l!=i}^{n}N(0, \sigma_2^2)\delta' e_l$

where $X_i$ is the randomly selected parent of offspring $X_i'$, and $e_l$ are the n −1 orthonormal bases that span the subspace perpendicular to $di$

# Gene Scanning Techniques
developed a number of gene scanning techniques as multiparent generalizations of n-point crossover
The algorithm contains two main procedures:


• A scanning strategy, which assigns to each selected parent a probability that the offspring will inherit the next component from that parent. The component under consideration is indicated by a marker.


• A marker update strategy, which updates the markers of parents to point to the next component of each parent.

# 12. Uniform Scanning 

# 13. Occurence-Based Scanning

# 14. Fitness-Based Scanning 

# 15. Diagonal CrossOver Operator 